<a href="https://colab.research.google.com/github/9-coding/PyTorch/blob/main/09-tensor_storage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensor Storage
Tensor 인스턴스의 실제 데이터가 저장되는 1D numerical array 관리
- 여러 인스턴스들이 같은 storage를 공유할 수 있음.
- cpu 또는 gpu 등의 memory의 실제 data가 저장된 contiguous block 관리.
- 다양한 shape의 Tensor 인스턴스들은 자신의 storage 인스턴스를 통해 data block을 관리함.

In [20]:
import torch

x = torch.randn(3,3)

print(x.storage())

 0.07993819564580917
 -0.6680573225021362
 1.4372423887252808
 -0.04802791029214859
 3.21359920501709
 -0.0708732083439827
 -1.4636471271514893
 -1.8654823303222656
 0.1301005333662033
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 9]


- 실제 data가 저장되는 memory를 가리키는 위치를 반환하는 메서드 가짐.

In [32]:
import torch

a = torch.tensor([range(i,i+3) for i in [0,3,6]])
print(a)
a_sub = a[1:,0] # 부분에 대한 view

# Tensor a 와 a_sub 의 contiguous 여부 확인.
print(a.is_contiguous(), a_sub.is_contiguous())

# Tensor a 와 a_sub 가 같은 메모리의 데이터를 공유하는지 여부.
print(a.storage().data_ptr() == a_sub.storage().data_ptr())

# Storage 인스턴스는 각 Tensor 인스턴스 별로 생성됨.
print(id(a.storage()) == id(a_sub.storage()))

# Tensor 인스턴스의 meta data
print(f"{a.shape=}, {a.size()=}, {a.stride()=}, {a.storage_offset()=}")
print(f"{a_sub.shape=}, {a_sub.size()=}, {a_sub.stride()=}, {a_sub.storage_offset()=}")

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])
True False
True
False
a.shape=torch.Size([3, 3]), a.size()=torch.Size([3, 3]), a.stride()=(3, 1), a.storage_offset()=0
a_sub.shape=torch.Size([2]), a_sub.size()=torch.Size([2]), a_sub.stride()=(3,), a_sub.storage_offset()=3


- `stride`: 각 축에서 index가 증가할 때 건너뛰어야하는 객체의 수.
- `storage_offset`: 현재 Tensor 인스턴스가 Storage가 관리하는 memory block에서 몇 번째 객체에서 시작하는지를 나타냄.  

## Contiguous 확인

In [22]:
a = torch.tensor([ range(i,i+3) for i in [0,3,6]])
print(a)
print(a.storage())
print(a.is_contiguous())

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])
 0
 1
 2
 3
 4
 5
 6
 7
 8
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 9]
True


In [23]:
a_sub = a[1:,0]
print(a_sub)
print(a_sub.storage())
print(a_sub.is_contiguous())
print(a.storage().data_ptr() == a_sub.storage().data_ptr())

tensor([3, 6])
 0
 1
 2
 3
 4
 5
 6
 7
 8
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 9]
False
True


## View / Reshape

view: instance가 contiguous한 경우에만 사용 가능.<br>
reshape: <br>
- contiguous한 경우 view와 같은 동작.
- 아닌 경우엔 contiguous() 메서드를 호출하고 view를 수행한 것에 해당함. (이 경우  underlying memory block이 새로 만들어짐.)
- 따라서 memory를 공유할 수도 아닐 수도 있음.


In [37]:
print('contiguous한 경우')
a_r = a.reshape((9,))
a_r[3] = 777
print('데이터 저장공간을 share함.')
print(a.storage().data_ptr() == a_r.storage().data_ptr())
print('둘 다 contiguous')
print(a.is_contiguous(), a_r.is_contiguous())
print('같은 인스턴스는 아님')
print(id(a.storage()) == id(a_r.storage()))
print('a_r: af.reshape')
print(a_r)
print('a: orginal')
print(a)

contiguous한 경우
데이터 저장공간을 share함.
True
둘 다 contiguous
True True
같은 인스턴스는 아님
False
a_r: af.reshape
tensor([  0,   1,   2, 777,   4,   5,   6,   7,   8])
a: orginal
tensor([[  0,   1,   2],
        [777,   4,   5],
        [  6,   7,   8]])


In [25]:
print('contiguous하지 않은 경우')
a_t = a.T
a_r = a_t.reshape((9,))
a_r[3] = 777
print('데이터 저장공간을 share하지 않음.')
print(a.storage().data_ptr() == a_r.storage().data_ptr())
print('a_r은 contiguous')
print(a_t.is_contiguous(), a_r.is_contiguous())
print('같은 인스턴스는 아님')
print(id(a.storage()) == id(a_sub.storage()))

contiguous하지 않은 경우
데이터 저장공간을 share하지 않음.
False
a_r은 contiguous
False True
같은 인스턴스는 아님
False


## Transpose / Permute
- underlying data 를 공유하여 Tensor 인스턴스의 meta data (stride 등)을 수정한다.
- 단, 이를 통해 얻어진 인스턴스에서는
contiguous는 깨지게 된다.
- contiguous 메서드 호출을 하여 해당 인스턴스의 meta data를 기반으로 Tensor 인스턴스를 새로 얻는 경우에는 새로운 underlying data가 만들어지게 된다.

In [35]:
a = torch.arange(9).reshape((3,3))
print(a)
a_t = torch.transpose(a,0,1)
print(a_t)
print('데이터 저장공간을 share함.')
print(a.storage().data_ptr() == a_t.storage().data_ptr())
print('transpose는 contiguous하지 않음')
print(a.is_contiguous(), a_t.is_contiguous())
print('같은 인스턴스는 아님')
print(id(a.storage()) == id(a_t.storage()))

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])
tensor([[0, 3, 6],
        [1, 4, 7],
        [2, 5, 8]])
데이터 저장공간을 share함.
True
둘 다 contiguous
True False
같은 인스턴스는 아님
False


In [27]:
print('a_t의 값을 바꾸면 a에 적용')

a_t[0,1] = 999
print(a_t)
print('-------')
print(a)
print('-------')
print(a_sub)

a_t의 값을 바꾸면 a에 적용
tensor([[  0, 999,   6],
        [  1,   4,   7],
        [  2,   5,   8]])
-------
tensor([[  0,   1,   2],
        [999,   4,   5],
        [  6,   7,   8]])
-------
tensor([999,   6])


In [28]:
act = a_t.contiguous()
print(act)
print(act.storage())
act[1] = 999
print(act)
print(a_t)

tensor([[  0, 999,   6],
        [  1,   4,   7],
        [  2,   5,   8]])
 0
 999
 6
 1
 4
 7
 2
 5
 8
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 9]
tensor([[  0, 999,   6],
        [999, 999, 999],
        [  2,   5,   8]])
tensor([[  0, 999,   6],
        [  1,   4,   7],
        [  2,   5,   8]])


In [29]:
a_t_r = a_t.reshape((9,)) # uncontiguous tensor에서는 새로 생성이 이루어짐.
print(a_t_r)
a_t_r[1] = 999
print(a_t_r)
print(a_t) # 달라짐.

tensor([  0, 999,   6,   1,   4,   7,   2,   5,   8])
tensor([  0, 999,   6,   1,   4,   7,   2,   5,   8])
tensor([[  0, 999,   6],
        [  1,   4,   7],
        [  2,   5,   8]])


In [30]:
print(a)
print('------')
print('permute axis-0 and axis-1 = transpose')
ap = a.permute([1,0]) # permute axis-0 and axis-1
print(ap)
print(a.storage().data_ptr() == ap.storage().data_ptr())
print(ap.is_contiguous())

tensor([[  0,   1,   2],
        [999,   4,   5],
        [  6,   7,   8]])
------
permute axis-0 and axis-1 = transpose
tensor([[  0, 999,   6],
        [  1,   4,   7],
        [  2,   5,   8]])
True
False


In [31]:
print(f"{a.shape=}, {a.size()=}, {a.stride()=}, {a.storage_offset()=}")
print(f"{a_t.shape=}, {a_t.size()=}, {a_t.stride()=}, {a_t.storage_offset()=}")
print(f"{a_sub.shape=}, {a_sub.size()=}, {a_sub.stride()=}, {a_sub.storage_offset()=}")

a.shape=torch.Size([3, 3]), a.size()=torch.Size([3, 3]), a.stride()=(3, 1), a.storage_offset()=0
a_t.shape=torch.Size([3, 3]), a_t.size()=torch.Size([3, 3]), a_t.stride()=(1, 3), a_t.storage_offset()=0
a_sub.shape=torch.Size([2]), a_sub.size()=torch.Size([2]), a_sub.stride()=(3,), a_sub.storage_offset()=3
